In [1]:
#coding=utf-8

# if PyMySQL not install before, install for manipulating mysql database
# %pip install pymysql

import time
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment, SoupStrainer

import pymysql


In [2]:
db_config = {'host':'localhost',
             'user':'root',
             'password':'mima123456',
             'database':'mysql',
             'charset':'utf8',
             'cursorclass': pymysql.cursors.DictCursor}

In [3]:
def connect_db():
    try:
        # connect to mysql database
        conn = pymysql.connect(**db_config)

    except mysql.connector.Error as e:
        print('DATABASE Connection Error Occur: ', e)

    return conn


In [4]:
# # 使用 cursor() 方法创建一个游标对象 cursor
db = connect_db()

cursor = db.cursor()

sql = "SELECT * FROM  movie_showing;"
cursor.execute(sql)
cursor.fetchall()

[{'title': 'Prince (Digital)',
  'AKA': '',
  'title_EN': 'Prince (Digital)',
  'title_CN': '',
  'poster_src': 'https://www.wecinemas.com.sg/Data/Movies/2078/158_1.jpg',
  'opening_date': '21 Oct 2022',
  'ratings': 'PG (Some Violence)',
  'duration': '150 mins',
  'director': 'Mr.Anudeep K.V. ',
  'cast': 'Sivakarthikeyan: Maria Ryaboshapka: Sathyaraj  and others ',
  'language': 'Tamil\xa0',
  'genre': 'Drama',
  'introduction': 'Anbu a social science teacher in the town of Devanakottai falls in with a young British woman Jessica who come to teach English in the same school as Anbu. The love leads to a comedy of errors due to the difference in the cultures and opinions of the people of the town and the couple themselves. Will anbu and jessica unite inspire o all of these discriminations and the differences that the society has forced upon them?A fun filled comedy of errors which celebrates the idea of harmony and peace in a world filled with discriminations and differences.',
  'sho

## extract data about movies now showing

In [4]:
def extract_movie_info(movie_info, end_url_list, html):
    html = requests.get(html)

    # get source page with content property
    # resolve with BeautifulSoap and transfer into BS class
    soup = BeautifulSoup(html.content,'lxml')

    table_list = soup.find("table",id='DataNowshowingListing')
#     print(table_list)
    table_movies = table_list.find("table",width='650')

    # find movie image info from DataNowshowingListing
    for tbl in table_movies.find_all('table', width="650"):
        tds = tbl.find_all('td', class_='mtitle-listing-txt')

        movie_info[tbl.select('img')[0]['title']] = {'title': tbl.select('img')[0]['alt'], 
                                                    'poster_src': 'https://www.wecinemas.com.sg/' + tbl.select('img')[0]['src'],
                                                    'title_EN': tds[0].text,
                                                     'title_CN': tds[1].text
                                                   }

        titles = tbl.find_all('td', class_='mdetail-listing-txt')
        infos = tbl.find_all('td', class_='mdetail-b-listing-txt')

        movie_info[tbl.select('img')[0]['title']][titles[0].text] = infos[0].text
        movie_info[tbl.select('img')[0]['title']][titles[2].text] = infos[1].text
        movie_info[tbl.select('img')[0]['title']][titles[4].text] = infos[2].text
        movie_info[tbl.select('img')[0]['title']]['introduction'] = titles[6].text


    # get all sub_url for detailed info
    for i in soup.find_all('span', class_='listing-but'):
        if i.text == 'INFO':
            item = i.find_all('a')
            for j in item:
                end_url = j.get('href')
                if end_url not in end_url_list:
                    end_url_list.append(j.get('href'))

    time.sleep(10)


In [5]:
#extract data from webpage
movie_info = {}
end_url_list = []

for i in range(1,3):
    extract_movie_info(movie_info, end_url_list, 'https://www.wecinemas.com.sg/listing.aspx?tab=nowshowing&Page={}'.format(i))


## extract data from info page

### extract movie related info including showtime (put into detailed_movie_info)

In [6]:
def extract_detailed_movie_info(movie_info, html):
    #extract data from webpage
    html = requests.get(html)

    #get source page with content property
    #resolve with BeautifulSoap and transfer into BS class
    info = BeautifulSoup(html.content,'lxml')

    dmovie_info = info.find("table",width='436')
    
    movie_title = dmovie_info.find_all('span', id='lblMovieTitle')[0].text
    
    if movie_title in movie_info.keys():
    
        movie_AKA = dmovie_info.find_all('span', id='lblMovieAKA')[0].text
        titles = dmovie_info.find_all('td', class_='mdetail-listing-txt')
        infos = dmovie_info.find_all('td', class_='mdetail-b-listing-txt')
        intro = dmovie_info.find_all('td', class_='content-details-txt')

        movie_info[movie_title]['AKA'] = movie_AKA
        movie_info[movie_title]['Opening Date'] = dmovie_info.find_all('span', id='lblMovieReleaseDate')[0].text
        movie_info[movie_title]['Director'] = dmovie_info.find_all('span', id='lblMovieDirector')[0].text
        movie_info[movie_title]['Cast'] = dmovie_info.find_all('span', id='lblMovieCast')[0].text
        movie_info[movie_title]['Ratings'] = dmovie_info.find_all('span', id='lblMovieRating')[0].text
        movie_info[movie_title]['Duration'] = dmovie_info.find_all('span', id='lblMovieRuntime')[0].text
        movie_info[movie_title]['Language'] = dmovie_info.find_all('span', id='lblMovieLanguage')[0].text
        movie_info[movie_title]['Genre'] = dmovie_info.find_all('span', id='lblMovieGenre')[0].text
        movie_info[movie_title]['introduction'] = dmovie_info.find_all('span', id='lblMovieSynopsis')[0].text

        showtime = dmovie_info.find_all('option')

        movie_info[movie_title]['showdate'] = []

        for st in showtime:
            movie_info[movie_title]['showdate'].append(st.text)

        movie_info[movie_title]['showtime'] = dmovie_info.find_all('div', class_="showtimes-but")[0].text


        rmovie_info = info.find("table",width='436')
        recommend_movie = []

        rec_imgs = rmovie_info.find_all('img')
        rec_titles = rmovie_info.find_all('td', class_='mtitle-details-txt')
        rec_dates = rmovie_info.find_all('td', class_='mdate-home-txt')

        for i in range(len(rec_imgs)):
            recommend_movie.append([rec_imgs[i]['src'], rec_titles[i].text, rec_dates[i].text])

        movie_info[movie_title]['recommended_movie'] = recommend_movie


In [7]:
detail_movie_html = []
base_url = 'https://www.wecinemas.com.sg{}'

for end_url in end_url_list:
    url = base_url.format(end_url)

    extract_detailed_movie_info(movie_info,url)


In [8]:
x = pd.DataFrame(movie_info).T.reset_index()
x = x.drop(['index'], axis=1)
x

,title,poster_src,title_EN,title_CN,Opening Date,Ratings,Duration,introduction,AKA,Director,Cast,Language,Genre,showdate,showtime,recommended_movie
0,Prince (Digital),https://www.wecinemas.com.sg/Data/Movies/2078/...,Prince (Digital),,21 Oct 2022,PG (Some Violence),150 mins,Anbu a social science teacher in the town of D...,,Mr.Anudeep K.V.,Sivakarthikeyan: Maria Ryaboshapka: Sathyaraj ...,Tamil,Drama,[26 Oct 2022],9:15PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
1,Sardar (Digital),https://www.wecinemas.com.sg/Data/Movies/2082/...,Sardar (Digital),,21 Oct 2022,PG13 (Some Violence),166 mins,TBA,,P.S. Mithran,karthi:rajisha vijayan:raashi Khanna and others.,Tamil,"Action, Drama","[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",5:25PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
2,Black Adam (First Class),https://www.wecinemas.com.sg/Data/Movies/2067/...,Black Adam (First Class),黑亚当,20 Oct 2022,PG13 (Some Violence 些许暴力画面),125 mins,"Nearly 5,000 years after he was bestowed with ...",黑亚当,Jaume Collet-Serra,"Dwayne Johnson, Sarah Shahi, Pierce Brosnan, A...",English with Chinese Subtitles,"Action, Sci-Fi, Fantasy","[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",5:00PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
3,Black Adam (Digital),https://www.wecinemas.com.sg/Data/Movies/1910/...,Black Adam (Digital),黑亚当,20 Oct 2022,PG13 (Some Violence 些许暴力画面),125 mins,Movie free listed from 27 October 2022Nearly 5...,黑亚当,Jaume Collet-Serra,"Dwayne Johnson, Sarah Shahi, Pierce Brosnan, A...",English with Chinese Subtitles,"Action, Sci-Fi, Fantasy","[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",4:00PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
4,Mrs Harris Goes to Paris (Digital),https://www.wecinemas.com.sg/Data/Movies/2058/...,Mrs Harris Goes to Paris (Digital),,20 Oct 2022,PG,116 mins,Movie free listed from 27 October 2022.A new a...,,Anthony Fabian,"Jason Isaacs, Lesley Manville, Isabelle Huppert",English with no subtitles,Drama,"[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",9:15PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
5,Halloween Ends (M18) (Digital),https://www.wecinemas.com.sg/Data/Movies/1991/...,Halloween Ends (M18) (Digital),月光光新慌慌：万圣结,13 Oct 2022,M18 (Violence and Gore 暴力及血腥画面),111 mins,Movie free listed from 20 October 2022.The sag...,月光光新慌慌：万圣结,David Gordon Green,"Judy Greer, Jamie Lee Curtis, Nick Castle, And...",English with Chinese Subtitles,"Horror, Thriller",[26 Oct 2022],4:00PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
6,Ticket to Paradise (Digital),https://www.wecinemas.com.sg/Data/Movies/2007/...,Ticket to Paradise (Digital),幸福入场券,6 Oct 2022,PG13 (Some Coarse Language 些许粗俗语言),104 mins,Movie free listed from 13 October 2022.A divor...,幸福入场券,OI Parker,"Julia Roberts, Kaitlyn Dever, Billie Lourd, Ge...",English with Chinese Subtitles,"Comedy, Romance",[26 Oct 2022],4:25PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
7,ONE PIECE FILM RED (Digital),https://www.wecinemas.com.sg/Data/Movies/2016/...,ONE PIECE FILM RED (Digital),,29 Sep 2022,PG13 (Some Violence 些许暴力画面),115 mins,Uta — the most beloved singer in the world.Her...,,Goro Taniguchi,Kaori Nazuka as Uta Mayumi Tanaka as Monkey D....,Japanese with English & Chinese Subtitles,Animation,"[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",6:45PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."


coming_soon_movies

In [9]:
def extrac_showing_movie_info(coming_movie_info, end_url_list, html):
    html = requests.get(html)

    # get source page with content property
    # resolve with BeautifulSoap and transfer into BS class
    soup = BeautifulSoup(html.content,'lxml')

    coming_table_list = soup.find("table",id='DataComingSoonListing')

    # find movie image info from DataNowshowingListing
    for tbl in coming_table_list.find_all('table', width="650"):
        tds = tbl.find_all('td', class_='mtitle-listing-txt')

        coming_movie_info[tbl.select('img')[0]['title']] = {'title': tbl.select('img')[0]['alt'], 
                                                    'poster_src': 'https://www.wecinemas.com.sg/' + tbl.select('img')[0]['src'],
                                                    'title_EN': tds[0].text,
                                                    'title_CN': tds[1].text
                                                   }

        titles = tbl.find_all('td', class_='mdetail-listing-txt')
        infos = tbl.find_all('td', class_='mdetail-b-listing-txt')

        coming_movie_info[tbl.select('img')[0]['title']][titles[0].text] = infos[0].text
        coming_movie_info[tbl.select('img')[0]['title']][titles[2].text] = infos[1].text
        coming_movie_info[tbl.select('img')[0]['title']][titles[4].text] = infos[2].text
        coming_movie_info[tbl.select('img')[0]['title']]['introduction'] = titles[6].text

    time.sleep(10)


In [10]:
#extract data from webpage
coming_movie_info = {}
end_url_list = []

for i in range(1,6):
    extrac_showing_movie_info(coming_movie_info, end_url_list, 'https://www.wecinemas.com.sg/listing.aspx?tab=comingsoon&Page={}'.format(i))


In [12]:
y = pd.DataFrame(coming_movie_info).T.reset_index()
y = y.drop(['index'], axis=1)
y

,title,poster_src,title_EN,title_CN,Opening Date,Ratings,Duration,introduction
0,Ajoomma (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2036/...,Ajoomma (NC16) (Digital),花路阿朱妈,27 Oct 2022,NC16 (Some Mature References 内容只宜十六岁及以上),90 mins,Produced by award-winning filmmaker Anthony Ch...
1,Come Back Home (Digital),https://www.wecinemas.com.sg/Data/Movies/2083/...,Come Back Home (Digital),搜救,27 Oct 2022,PG (SOME INTENSE SEQUENCES 些许惊险画面),102 mins,Movie free listed from 3 November 2022.\n\nIn ...
2,Home Coming (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2079/...,Home Coming (NC16) (Digital),万里归途,27 Oct 2022,NC16 (Violence 暴力画面),138 mins,Movie free listed from 1 November 2022\n\nWar ...
3,Prey For The Devil (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2080/...,Prey For The Devil (NC16) (Digital),魔鬼之光,27 Oct 2022,NC16 (Horror 惊悚画面),93 mins,Movie free listed from 3 November 2022.\n\nIn ...
4,Marvel Studios’ Black Panther: Wakanda Forever...,https://www.wecinemas.com.sg/Data/Movies/2025/...,Marvel Studios’ Black Panther: Wakanda Forever...,黑豹 2,10 Nov 2022,To Be Advised,To Be Advised,Movie free listed from 24 November 2022.\n\nA ...
5,The Menu (Digital),https://www.wecinemas.com.sg/Data/Movies/2032/...,The Menu (Digital),菜单,17 Nov 2022,To Be Advised,107 mins,Movie free listed from 1 December 2022.\n\nA y...
6,Bones and All (Digital),https://www.wecinemas.com.sg/Data/Movies/2066/...,Bones and All (Digital),骨肉的总和,24 Nov 2022,To Be Advised (TBA),131 mins,Maren Yearly is a young woman who wants the sa...
7,Disney's Strange World (Digital),https://www.wecinemas.com.sg/Data/Movies/1989/...,Disney's Strange World (Digital),,24 Nov 2022,To Be Advised,To Be Advised,The legendary Clades are a family of explorers...
8,She Said (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2000/...,She Said (NC16) (Digital),她有话要说,24 Nov 2022,NC16 (Mature Content 内容只宜十六岁及以上),129 mins,Movie free listed from 1 December 2022.\n\nNew...
9,"Lyle, Lyle, Crocodile (Digital)",https://www.wecinemas.com.sg/Data/Movies/2022/...,"Lyle, Lyle, Crocodile (Digital)",鳄鱼歌王,1 Dec 2022,PG,107 mins,Movie free listed from 08 December 2022.\n\nFe...


In [18]:
y['Ratings'] = y['Ratings'].apply(lambda mv: 'NC16' if mv=='To Be Advised' else mv)
y['Duration'] = y['Duration'].apply(lambda mv: '105 mins' if mv=='To Be Advised' else mv)
y


,title,poster_src,title_EN,title_CN,Opening Date,Ratings,Duration,introduction
0,Ajoomma (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2036/...,Ajoomma (NC16) (Digital),花路阿朱妈,27 Oct 2022,NC16 (Some Mature References 内容只宜十六岁及以上),90 mins,Produced by award-winning filmmaker Anthony Ch...
1,Come Back Home (Digital),https://www.wecinemas.com.sg/Data/Movies/2083/...,Come Back Home (Digital),搜救,27 Oct 2022,PG (SOME INTENSE SEQUENCES 些许惊险画面),102 mins,Movie free listed from 3 November 2022.\n\nIn ...
2,Home Coming (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2079/...,Home Coming (NC16) (Digital),万里归途,27 Oct 2022,NC16 (Violence 暴力画面),138 mins,Movie free listed from 1 November 2022\n\nWar ...
3,Prey For The Devil (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2080/...,Prey For The Devil (NC16) (Digital),魔鬼之光,27 Oct 2022,NC16 (Horror 惊悚画面),93 mins,Movie free listed from 3 November 2022.\n\nIn ...
4,Marvel Studios’ Black Panther: Wakanda Forever...,https://www.wecinemas.com.sg/Data/Movies/2025/...,Marvel Studios’ Black Panther: Wakanda Forever...,黑豹 2,10 Nov 2022,NC16,105 mins,Movie free listed from 24 November 2022.\n\nA ...
5,The Menu (Digital),https://www.wecinemas.com.sg/Data/Movies/2032/...,The Menu (Digital),菜单,17 Nov 2022,NC16,107 mins,Movie free listed from 1 December 2022.\n\nA y...
6,Bones and All (Digital),https://www.wecinemas.com.sg/Data/Movies/2066/...,Bones and All (Digital),骨肉的总和,24 Nov 2022,To Be Advised (TBA),131 mins,Maren Yearly is a young woman who wants the sa...
7,Disney's Strange World (Digital),https://www.wecinemas.com.sg/Data/Movies/1989/...,Disney's Strange World (Digital),,24 Nov 2022,NC16,105 mins,The legendary Clades are a family of explorers...
8,She Said (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2000/...,She Said (NC16) (Digital),她有话要说,24 Nov 2022,NC16 (Mature Content 内容只宜十六岁及以上),129 mins,Movie free listed from 1 December 2022.\n\nNew...
9,"Lyle, Lyle, Crocodile (Digital)",https://www.wecinemas.com.sg/Data/Movies/2022/...,"Lyle, Lyle, Crocodile (Digital)",鳄鱼歌王,1 Dec 2022,PG,107 mins,Movie free listed from 08 December 2022.\n\nFe...


In [19]:
dd = pd.concat([x,y]).reset_index()
dd

,index,title,poster_src,title_EN,title_CN,Opening Date,Ratings,Duration,introduction,AKA,Director,Cast,Language,Genre,showdate,showtime,recommended_movie
0,0,Prince (Digital),https://www.wecinemas.com.sg/Data/Movies/2078/...,Prince (Digital),,21 Oct 2022,PG (Some Violence),150 mins,Anbu a social science teacher in the town of D...,,Mr.Anudeep K.V.,Sivakarthikeyan: Maria Ryaboshapka: Sathyaraj ...,Tamil,Drama,[26 Oct 2022],9:15PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
1,1,Sardar (Digital),https://www.wecinemas.com.sg/Data/Movies/2082/...,Sardar (Digital),,21 Oct 2022,PG13 (Some Violence),166 mins,TBA,,P.S. Mithran,karthi:rajisha vijayan:raashi Khanna and others.,Tamil,"Action, Drama","[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",5:25PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
2,2,Black Adam (First Class),https://www.wecinemas.com.sg/Data/Movies/2067/...,Black Adam (First Class),黑亚当,20 Oct 2022,PG13 (Some Violence 些许暴力画面),125 mins,"Nearly 5,000 years after he was bestowed with ...",黑亚当,Jaume Collet-Serra,"Dwayne Johnson, Sarah Shahi, Pierce Brosnan, A...",English with Chinese Subtitles,"Action, Sci-Fi, Fantasy","[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",5:00PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
3,3,Black Adam (Digital),https://www.wecinemas.com.sg/Data/Movies/1910/...,Black Adam (Digital),黑亚当,20 Oct 2022,PG13 (Some Violence 些许暴力画面),125 mins,Movie free listed from 27 October 2022Nearly 5...,黑亚当,Jaume Collet-Serra,"Dwayne Johnson, Sarah Shahi, Pierce Brosnan, A...",English with Chinese Subtitles,"Action, Sci-Fi, Fantasy","[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",4:00PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
4,4,Mrs Harris Goes to Paris (Digital),https://www.wecinemas.com.sg/Data/Movies/2058/...,Mrs Harris Goes to Paris (Digital),,20 Oct 2022,PG,116 mins,Movie free listed from 27 October 2022.A new a...,,Anthony Fabian,"Jason Isaacs, Lesley Manville, Isabelle Huppert",English with no subtitles,Drama,"[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",9:15PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
5,5,Halloween Ends (M18) (Digital),https://www.wecinemas.com.sg/Data/Movies/1991/...,Halloween Ends (M18) (Digital),月光光新慌慌：万圣结,13 Oct 2022,M18 (Violence and Gore 暴力及血腥画面),111 mins,Movie free listed from 20 October 2022.The sag...,月光光新慌慌：万圣结,David Gordon Green,"Judy Greer, Jamie Lee Curtis, Nick Castle, And...",English with Chinese Subtitles,"Horror, Thriller",[26 Oct 2022],4:00PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
6,6,Ticket to Paradise (Digital),https://www.wecinemas.com.sg/Data/Movies/2007/...,Ticket to Paradise (Digital),幸福入场券,6 Oct 2022,PG13 (Some Coarse Language 些许粗俗语言),104 mins,Movie free listed from 13 October 2022.A divor...,幸福入场券,OI Parker,"Julia Roberts, Kaitlyn Dever, Billie Lourd, Ge...",English with Chinese Subtitles,"Comedy, Romance",[26 Oct 2022],4:25PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
7,7,ONE PIECE FILM RED (Digital),https://www.wecinemas.com.sg/Data/Movies/2016/...,ONE PIECE FILM RED (Digital),,29 Sep 2022,PG13 (Some Violence 些许暴力画面),115 mins,Uta — the most beloved singer in the world.Her...,,Goro Taniguchi,Kaori Nazuka as Uta Mayumi Tanaka as Monkey D....,Japanese with English & Chinese Subtitles,Animation,"[26 Oct 2022, 27 Oct 2022, 28 Oct 2022, 29 Oct...",6:45PM,"[[/Data/Movies/2025/158_1.jpg, Marvel Studios’..."
8,0,Ajoomma (NC16) (Digital),https://www.wecinemas.com.sg/Data/Movies/2036/...,Ajoomma (NC16) (Digital),花路阿朱妈,27 Oct 2022,NC16 (Some Mature References 内容只宜十六岁及以上),90 mins,Produced by award-winning filmmaker Anthony Ch...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,Come Back Home (Digital),https://www.wecinemas.com.sg/Data/Movies/2083/...,Come Back Home (Digital),搜救,27 Oct 2022,PG (SOME INTENSE SEQUENCES 些许惊险画面),102 mins,Movie free listed from 3 November 2022.\n\nIn ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
dd.to_csv('movie_data.csv')

## connect to database

In [14]:
# connect to database
db = connect_db()

# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
 



### showing movie info

In [156]:
# create tables to store theatre data----- movie_info

sql="""
    CREATE TABLE movie_showing (
          title  VARCHAR(1000) NOT NULL,
          AKA VARCHAR(1000),
          title_EN  VARCHAR(1000),
          title_CN  VARCHAR(1000),
          poster_src VARCHAR(1000),
          opening_date VARCHAR(100),
          ratings VARCHAR(200),
          duration VARCHAR(100),
          director VARCHAR(500),
          cast VARCHAR(1000),
          language VARCHAR(200),
          genre VARCHAR(200),
          introduction VARCHAR(1000),
          showdate VARCHAR(2000),
          showtime VARCHAR(200),
          recommended_movie VARCHAR(1000)
          )
    """


cursor.execute(sql)
db.commit()


In [16]:
sql = 'insert into movie_showing values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

for mv_info in movie_info.values():
    data = (mv_info['title'], mv_info['AKA'], mv_info['title_EN'], mv_info['title_CN'], 
            mv_info['poster_src'], mv_info['Opening Date'], mv_info['Ratings'], mv_info['Duration'], 
            mv_info['Director'], mv_info['Cast'], mv_info['Language'], mv_info['Genre'], 
            mv_info['introduction'], str(mv_info['showdate']), str(mv_info['showtime']), str(mv_info['recommended_movie']))

    cursor.execute(sql, data)

db.commit()


### create manager info

In [ ]:
# create tables to store theatre data----- movie_info

sql="""
    CREATE TABLE manager_info (
          phone  VARCHAR(100) NOT NULL,
          password VARCHAR(100) NOT NULL
          )
    """


cursor.execute(sql)
db.commit()


In [8]:
manager_info = [['12345678', 'manager01'], ['12345679', 'manager02'], ['12345670', 'manager03']]

sql = 'insert into manager_info values (%s, %s)'


for manager in manager_info:
    info = (manager[0], manager[1])
    cursor.execute(sql, info)


db.commit()


## API for UI to fetch data and manage database

In [250]:
# fetch data request API
# example: table: 'movie_showing', length: 2
def on_data_fetch_request(table, length, *columns):
    if not columns:
        columns = '*'
        sql = 'SELECT {} FROM {}'.format(columns, table)
    else:
        columns = ','.join(list(columns[0]))
        sql = 'SELECT {} FROM {}'.format(columns, table)
    if length:
        sql += ' LIMIT {}'.format(length)


    db = connect_db()
    cur = db.cursor()
    cur.execute(sql)
    return cur.fetchall()

    

In [26]:
# API test
# rt = on_data_fetch_request('movie_showing', 2)

In [22]:
# order_type: insert, delete, update
# insert order example: insert(order_type) into {table} {condition[0](as columns)} values {condition[1](as values)}
    # condition like: [(col1, col2, col3,...), (val1, val2, val3,...)]
# delete order example: delete(order_type) * from {table} where {condition}
    # condition like: {col1:val1, col2:val2,...}
# update order example: update(order_type) {table} set {col='' where col1='' and col2=''}
    # condition like: {'set': {}, 'where': {}}

def on_manage_db_request(order_type, table, condition):
    sql = ''
    if order_type.lower() == 'insert':
        sql = 'INSERT INTO {} {} VAlUES {}'.format(table, condition[0], condition[1])
    elif order_type.lower() == 'delete':
        con = ''
        for key, val in condition.items():
            con += f" {key}={val} and "
        sql = 'DELETE * from {} where {}'.format(table, con[:-5])
    elif order_type.lower() == 'update':
        set_con = ''
        for key, val in condition['set'].items():
            set_con += f" {key}={val} and "
        
        where_con = ''
        for key, val in condition['where'].items():
            where_con += f" {key}={val} and "
        
        sql = 'UPDATE {} SET {} where {}'.format(table, set_con[:-5], where_con[:-5])
    else: 
        raise ValueError('Invalid order_type!!!')
    
    db = connect_db()
    cur = db.cursor()
    cur.execute(sql)
    return cur.fetchall()



In [25]:
# API test:
# on_manage_db_request('insert', 'movie', [('col1','col2','col3'), ('val1', 'val2', 'val3')])

# on_manage_db_request('delete', 'movie', {'col1':'val1', 'col2':'val2', 'col3':'val3'})

# on_manage_db_request('update', 'movie', {'set':{'col1':'val1', 'col2':'val2', 'col3':'val3'}, 'where':{'col4':'val4','col5':'val5', 'col6':'val6'}})


INSERT INTO movie ('col1', 'col2', 'col3') VAlUES ('val1', 'val2', 'val3')
DELETE * from movie where  col1=val1 and  col2=val2 and  col3=val3
UPDATE movie SET  col1=val1 and  col2=val2 and  col3=val3 where  col4=val4 and  col5=val5 and  col6=val6


In [11]:
def on_query_manager_info(phone):
    sql = "SELECT password FROM manager_info where phone = '{}'".format(phone)
    
    db = connect_db()
    cur = db.cursor()
    cur.execute(sql)
    return cur.fetchall()


In [12]:
# API test
# on_query_manager_info('12345678')


[{'password': 'manager01'}]

### coming movie info

In [41]:
# # create tables to store theatre data----- coming movie info

# sql="""
#     CREATE TABLE coming_movie (
#           title  VARCHAR(200) NOT NULL,
#           title_EN  VARCHAR(200),
#           title_CN  VARCHAR(200),
#           poster_src VARCHAR(200),
#           opening_date VARCHAR(20),
#           rating VARCHAR(200),
#           duration VARCHAR(20),
#           introduction VARCHAR(2000))
#     """

# cursor.execute(sql)
# db.commit()


# insert data into coming_movie table
# sql = 'insert into coming_movie values(%s, %s, %s, %s, %s, %s, %s, %s)'

# for mv_info in coming_movie_info.values():
#     data = (mv_info['title'], mv_info['title_EN'], mv_info['title_CN'], mv_info['poster_src'], 
#          mv_info['Opening Date'], mv_info['Ratings'], mv_info['Duration'], mv_info['introduction'])

#     cursor.execute(sql, data)

# db.commit()


### detailed movie info

In [44]:
# # create tables to store theatre data----- detailed movie info

# sql="""
#     CREATE TABLE detailed_movie_info (
#           title  VARCHAR(200) NOT NULL,
#           AKA  VARCHAR(200),
#           opening_date VARCHAR(20),
#           director VARCHAR(200),
#           cast VARCHAR(200),
#           rating VARCHAR(200),
#           duration VARCHAR(20),
#           language VARCHAR(200),
#           genre VARCHAR(50),
#           introduction VARCHAR(2000),
#           showdate VARCHAR(100),
#           showtime VARCHAR(50),
#           recommended_movie VARCHAR(2000)
#           )
#     """

# cursor.execute(sql)
# db.commit()


# insert data into detailed_movie_info table
# sql = 'insert into detailed_movie_info values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

# for key, value in detailed_movie_info.items():
#     data = (key, value['AKA'], value['Opening Date'], value['Director'], value['Cast'], 
#          value['Ratings'], value['Duration'], value['Language'], value['Genre'],
#          value['introduction'], str(value['showdate']), value['showtime'], str(value['recommended_movie']))

#     cursor.execute(sql, data)

# db.commit()


## extract data about coming movies

In [ ]:
# coming_table_list = soup.find("table",id='DataComingSoonListing')

# # find movie image info from DataNowshowingListing
# coming_movie_info = {}
# for tbl in coming_table_list.find_all('table', width="650"):
#     tds = tbl.find_all('td', class_='mtitle-listing-txt')

#     coming_movie_info[tbl.select('img')[0]['title']] = {'title': tbl.select('img')[0]['alt'], 
#                                                 'poster_src': tbl.select('img')[0]['src'],
#                                                 'title_EN': tds[0].text,
#                                                 'title_CN': tds[1].text
#                                                }
    
#     titles = tbl.find_all('td', class_='mdetail-listing-txt')
#     infos = tbl.find_all('td', class_='mdetail-b-listing-txt')

#     coming_movie_info[tbl.select('img')[0]['title']][titles[0].text] = infos[0].text
#     coming_movie_info[tbl.select('img')[0]['title']][titles[2].text] = infos[1].text
#     coming_movie_info[tbl.select('img')[0]['title']][titles[4].text] = infos[2].text
#     coming_movie_info[tbl.select('img')[0]['title']]['introduction'] = titles[6].text

